In [1]:
import json
from glob import glob
import numpy as np
import re
from collections import defaultdict

In [2]:
# Opening JSON file
def extract_numbers(test_string):
    test_string = test_string.replace('<h-l-1>', '')
    test_string = test_string.replace('</h-l-1>', '')

    test_string = test_string.replace('<h-l-2>', '')

    test_string = test_string.replace('</h-l-2>', '')

    test_string = test_string.replace('<h-l-3>', '')
    test_string = test_string.replace('</h-l-3>', '')

    numbers = re.findall(r'\d+', str(test_string))
    return numbers

batches = ['0', '1']
usefuleness, conf, like = [], [], []
reliance = []
divergence, div_easy, original = [], [],[]
divergence_type = []
a_ =defaultdict(list)
time_without, time_with = [],[]
for batch in batches: 
    for f_name in glob(f'responses/ced_gpt/gpt_lit_en_pt_ced_{batch}/*.json'):
        critical_hard_true, critical_hard_false, critical_easy_true, critical_easy_false = 0,0,0,0
        print(f_name)
        #usefuleness, conf, like = [], [], []
        reliance = []
        div_easy =[]
        divergence, original = [], []
        divergence_type = []
        time_without, time_with = [],[]
        f = open(f_name)
        data = json.load(f)
        try:
            if data['language'] == 'Limited':
                continue
        except:
            a=1
        
        #try:
        #    if data['language_pt'] != 'native\t' and data['language'] != 'native\t':
        #        continue
        #except:
        #    a=1

        if data['with_hl'] == 'false':
            a=1
            #continue
        print(data['with_hl'])
        #print(data['feedback'])
        #print(data['notes28'])
        #continue

        conf.append(float(data['r28']))
        #print(data['with_hl'])
        #print(data['totalTimeInMinutes'])
        if data['with_hl'] == 'false':
            time_with.append(data['totalTimeInMinutes'])
        else:
            time_without.append(data['totalTimeInMinutes'])

        #print(data['r28'])
        #if data['r28'] != '1':
        #    continue
        try:
            usefuleness.append(float(data['h28']))
            #print(data['h28'])
        except:
            a = 1
        #    continue
        
        try:
            if data['with_hl'] == 'true':
                like.append(float(data['l28']))
        except:
            a=1

        #divergence, paraphrase, original = [], [],[]
        #divergence_type = []
        inds = [a for a in data['sample_indices'].split(',')]
        for id_, ind_ in enumerate(inds):
            ind = str(ind_)
            with open(f'static/examples/for_user_study/gpt_lit_en_pt_ced_{batch}/meta_lbl_{ind}.txt', 'r') as meta:
                meta = meta.readlines()[0]

                try:
                    anno = data[f't{id_+1}']
                except:
                    print('Missing')
                    print(meta)
                    continue

                #print(f'\nGold: {meta}')
                #print(f'Anno: {anno}')
                #if data['atncheck1'] != data['class2']:
                #    print(data['atncheck1'])
                #    print(data['class2'])
                #    print('Failed first attention check')
                #    continue

                #if data['atncheck2'] != data['t7']:
                #    print('Failed second attention check')
                #    continue
                #continue
                #if True:
                if meta=='ced_gpt':#  anno=='no':
                    
                    with open(f'static/examples/for_user_study/gpt_lit_en_pt_ced_{batch}/src_{ind}.txt', 'r') as src:
                        src = src.readlines()[0]
                        src_num = extract_numbers(src)
                        #print(f'\n{src}')

                    with open(f'static/examples/for_user_study/gpt_lit_en_pt_ced_{batch}/tgt_{ind}.txt', 'r') as tgt:
                        tgt = tgt.readlines()[0]
                        tgt_num = extract_numbers(tgt)
                        #print(f'\n{tgt}')
                    #print('-'*100)
                #print(tgt_num)
                
                #if set(tgt_num)!=set(src_num):
                #    print(src)
                #    print(tgt)
                #    continue
                if meta =='ced_gpt':
                    #print(type_)
                    #print(src)
                    #print(tgt)
                    #print('-'*100)
                    a=1

                #if anno == 'yes' or data[f'class{id_+1}'] !='N/A':
                type_ = data[f'class{id_+1}']
                if 'Major' in type_:# or 'Minor' in type_:
                    anno_corrected = 'yes'
                    if data['with_hl'] == 'false': 
                        if meta == 'ced_gpt':
                            critical_hard_false +=1
                        elif meta =='ced':
                            critical_easy_false +=1
                    else:
                        if meta == 'ced':
                            critical_easy_true +=1
                        elif meta =='ced_gpt':
                            critical_hard_true +=1 
                            
                            
                elif 'Minor' in type_:
                    anno_corrected = 'no'
                    if data['with_hl'] == 'false': 
                        if meta == 'ced':
                            critical_easy_false +=1   
                    else:
                        if meta =='ced':
                            critical_easy_true +=1

        
                else:
                    anno_corrected = 'no'
                #print(f'{meta}\t{anno}\t{type_}')
                if meta == 'ced_gpt':# or meta == 'ced':
                    #print(f'{meta}\t{anno}\t{type_}')
                    divergence.append(anno_corrected)
                    divergence_type.append(data[f'class{id_+1}'])
                elif meta == 'ced':
                    #print(f'{meta}\t{anno}\t{type_}')
                    div_easy.append(anno_corrected)
                    #continue
                else:
                    #print(f'{meta}\t{anno}\t{type_}')
                    original.append(anno_corrected)

                    
        d_yes, d = divergence.count('yes'), len(divergence)
        #print(f'Critical Hard: {d_yes} out of {d}')
        
        d_yes, d = div_easy.count('yes'), len(div_easy)
        #print(f'Critical Easy: {d_yes} out of {d}')
        
        o_no, o = original.count('no'), len(original)
        #print(f'Original: {o_no} out of {o}')

        tp = d_yes
        fp = original.count('yes')
        fn = divergence.count('no')

        precision = tp/(tp+fp)
        recall = tp/(tp+fn)
        #f1 = 2*(precision*recall)/(precision+recall)
        #over_rel = paraphrase.count('yes')
        #app_rel = (original.count('no') + divergence.count('yes'))/ (len(divergence) + len(original))
        u = np.mean(usefuleness)
        c = np.mean(conf)
        l = np.mean(like)

        #print(f'Precision: {precision}')
        #print(f'Recall: {recall}')
        #print(f'F1: {f1}')
        #print(f'Ap-reliance: {app_rel}')
        #print(f'Over-reliance: {over_rel}')

        #print(f'Self-confidence: {c}')
        if data['with_hl'] == 'false': 
            recall = critical_hard_false/5
            print(f'Critical Hard: {critical_hard_false}')
            print(f'Critical Easy: {critical_easy_false}')
        else:
            recall = critical_hard_true/5
            print(f'Critical Hard: {critical_hard_true}')
            print(f'Critical Easy: {critical_easy_true}')
            #print(f'Usefuleness: {u}')
            #print(f'Like to Use: {l}')
        try:
            en = data['language']
            print(f'EN: {en}')
        except:
            a =1
        pt = data['language_pt']
        print(f'PT: {pt}')
        print('\n')
        a_[data['with_hl']].append(recall)

responses/ced_gpt/gpt_lit_en_pt_ced_0/20230521-173723.json
true
Critical Hard: 3
Critical Easy: 5
EN: full
PT: native


responses/ced_gpt/gpt_lit_en_pt_ced_0/20230525-111906.json
true
Critical Hard: 3
Critical Easy: 5
EN: Professional
PT: native


responses/ced_gpt/gpt_lit_en_pt_ced_0/20230521-194230.json
false
Critical Hard: 2
Critical Easy: 5
EN: native
PT: native


responses/ced_gpt/gpt_lit_en_pt_ced_0/20230525-164631.json
false
Critical Hard: 3
Critical Easy: 3
EN: full
PT: Professional


responses/ced_gpt/gpt_lit_en_pt_ced_0/20230521-192517.json
false
Critical Hard: 3
Critical Easy: 5
EN: full
PT: native


responses/ced_gpt/gpt_lit_en_pt_ced_0/20230525-125038.json
true
Critical Hard: 4
Critical Easy: 5
EN: full
PT: native


responses/ced_gpt/gpt_lit_en_pt_ced_0/20230525-171737.json
true
Critical Hard: 4
Critical Easy: 5
EN: full
PT: native


responses/ced_gpt/gpt_lit_en_pt_ced_0/20230525-124444.json
false
Critical Hard: 3
Critical Easy: 5
EN: Professional
PT: native


responses/ce

KeyError: 'r28'

In [ ]:
np.mean(a_['true'])

In [ ]:
np.std(a_['true'])

In [ ]:
np.mean(a_['false'])

In [ ]:
np.std(a_['false'])

In [3]:
# Opening JSON file
batches =['0', '1']
import collections
from collections import defaultdict
divergence, paraphrase, original = [], [],[]
divergence_type = []
time_without, time_with = [],[]
with_, without_, gold_ = defaultdict(list), defaultdict(list),  defaultdict(str)
with_type, without_type = defaultdict(list),  defaultdict(list)
for batch in batches:
    for f_name in glob(f'responses/ced_gpt/gpt_lit_en_pt_ced_{batch}/*.json'):


        f = open(f_name)
        data = json.load(f)
        try:
            if data['language'] == 'Limited':
                continue
        except:
            a=1
        #if data['with_hl'] == 'false':
        #    continue

        #print(data['with_hl'])
        #print(data['totalTimeInMinutes'])
        if data['with_hl'] == 'true':
            time_with.append(data['totalTimeInMinutes'])
        else:
            time_without.append(data['totalTimeInMinutes'])


        #try:
        #    print(data['h28'])
        #except:
        #    continue

        #divergence, paraphrase, original = [], [],[]
        #divergence_type = []
        inds = [a for a in data['sample_indices'].split(',')]
        for id_, ind_ in enumerate(inds):
            ind = str(ind_)
            with open(f'static/examples/for_user_study/gpt_lit_en_pt_ced_{batch}/meta_lbl_{ind}.txt', 'r') as meta:
                meta = meta.readlines()[0]

                try:
                    anno = data[f't{id_+1}']
                except:
                    print('Missing')
                    continue

                #print(f'\nGold: {meta}')
                #print(f'Anno: {anno}')
                #if data['atncheck1'] != data['class2']:
                #    print('Failed first attention check')

                #if data['atncheck2'] != data['t7']:
                #    print('Failed second attention check')

                type_ = data[f'class{id_+1}']
                if 'Major' in type_:# or 'Minor' in type_:     
                    anno_corrected = 'yes'
                else:
                    anno_corrected = 'no'
                #anno_corrected = anno

                if meta == 'ced_gpt':
                    divergence.append(anno_corrected)
                    divergence_type.append(data[f'class{id_+1}'])
                else:
                    #continue
                    original.append(anno_corrected)

                if data['with_hl'] == 'true':
                    with_[f'{ind}_{batch}_{f_name}'].append(anno_corrected)
                    with_type[f'{ind}_{batch}_{f_name}'].append(data[f'class{id_+1}'])
                else:
                    without_[f'{ind}_{batch}_{f_name}'].append(anno_corrected)
                    without_type[f'{ind}_{batch}_{f_name}'].append(data[f'class{id_+1}'])


                #print(f'{ind} {meta}')
                gold_[f'{ind}_{batch}_{f_name}'] = meta

                #if meta == 'chatGPT_paraphrase' and anno=='yes':
                #    with open(f'static/examples/for_user_study/flores_gpt_paraphrase_fr/src_{ind}.txt', 'r') as src:
                #        src = src.readlines()[0]
                #        print(f'\n{src}')

                #    with open(f'static/examples/for_user_study/flores_gpt_paraphrase_fr/tgt_{ind}.txt', 'r') as tgt:
                #        tgt = tgt.readlines()[0]
                #        print(f'\n{tgt}')       

Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing


In [4]:
all_, correct = 0  , 0
for key, value in with_.items():
    if gold_[key] == 'ced_gpt':
        all_+=1
        if value[0] =='yes':
            correct+=1
print('With Hard ' + str(correct/all_))

all_, correct = 0  , 0
for key, value in without_.items():
    if gold_[key] == 'ced_gpt':
        all_+=1
        if value[0] =='yes':
            correct+=1
print('Without Hard ' + str(correct/all_))

all_, correct = 0  , 0
for key, value in with_.items():
    if gold_[key] == 'ced':
        all_+=1
        if value[0] =='yes':
            correct+=1
print('\nWith Easy ' + str(correct/all_))

all_, correct = 0  , 0
for key, value in without_.items():
    if gold_[key] == 'ced':
        all_+=1
        if value[0] =='yes':
            correct+=1
print('Without Easy ' + str(correct/all_))

all_, correct = 0  , 0
for key, value in with_.items():
    if gold_[key] == 'ced' or gold_[key] == 'ced_gpt' :
        all_+=1
        if value[0] =='yes':
            correct+=1
print('\nWith ALL ' + str(correct/all_))

all_, correct = 0  , 0
for key, value in without_.items():
    if gold_[key] == 'ced'  or gold_[key] == 'ced_gpt' :
        all_+=1
        if value[0] =='yes':
            correct+=1
print('Without ALL ' + str(correct/all_))


tp, fp, fn = 0, 0, 0
for key, value in without_.items():
    if gold_[key] == 'ced_gpt' and value[0] == 'yes':
        tp +=1
    elif gold_[key] == 'ced' and  value[0] == 'yes':
        tp +=1
            
    elif gold_[key] == 'ced_gpt' and  value[0] == 'no':
        fn +=1
    elif gold_[key] == 'ced' and  value[0] == 'no':
        fn +=1
            
    elif gold_[key] != 'ced_gpt' and  value[0] == 'yes':
        fp +=1
    elif gold_[key] != 'ced' and  value[0] == 'yes':
        fp +=1

precision_without = tp/(tp+fp)
recall_without = tp/(tp+fn)
f1_without = 2*(precision_without*recall_without)/(precision_without+recall_without)
    
tp, fp, fn = 0, 0, 0
for key, value in with_.items():
    if gold_[key] == 'ced_gpt' and value[0] == 'yes':
        tp +=1
    elif gold_[key] == 'ced' and  value[0] == 'yes':
        tp +=1
            
    elif gold_[key] == 'ced_gpt' and  value[0] == 'no':
        fn +=1
    elif gold_[key] == 'ced' and  value[0] == 'no':
        fn +=1
            
    elif gold_[key] != 'ced_gpt' and  value[0] == 'yes':
        fp +=1
    elif gold_[key] != 'ced' and  value[0] == 'yes':
        fp +=1

precision_with = tp/(tp+fp)
recall_with = tp/(tp+fn)
f1_with = 2*(precision_with*recall_with)/(precision_with+recall_with)

print('\nWITH')
print(f'Precision: {precision_with}')
print(f'Recall: {recall_with}')
print(f'F1: {f1_with}')

print('\nWITHOUT')
print(f'Precision: {precision_without}')
print(f'Recall: {recall_without}')
print(f'F1: {f1_without}')

With Hard 0.71
Without Hard 0.5684210526315789

With Easy 0.57
Without Easy 0.5473684210526316

With ALL 0.64
Without ALL 0.5578947368421052

WITH
Precision: 0.8951048951048951
Recall: 0.64
F1: 0.7463556851311953

WITHOUT
Precision: 0.8833333333333333
Recall: 0.5578947368421052
F1: 0.6838709677419355


In [5]:
class_ = 'ced'
counter_ = 'ced_gpt'
tp, fp, fn = 0, 0, 0
for key, value in without_.items():
    if gold_[key] == counter_:
        continue
    if gold_[key] == class_ and value[0] == 'yes':
        tp +=1
    elif gold_[key] == class_ and  value[0] == 'no':
        fn +=1            
    elif gold_[key] != class_ and  value[0] == 'yes':
        fp +=1


precision_without = tp/(tp+fp)
recall_without = tp/(tp+fn)
f1_without = 2*(precision_without*recall_without)/(precision_without+recall_without)
    
tp, fp, fn = 0, 0, 0
for key, value in with_.items():
    if gold_[key] == counter_:
        continue
    if gold_[key] == class_ and value[0] == 'yes':
        tp +=1            
    elif gold_[key] == class_ and  value[0] == 'no':
        fn +=1            
    elif gold_[key] != class_ and  value[0] == 'yes':
        fp +=1


precision_with = tp/(tp+fp)
recall_with = tp/(tp+fn)
f1_with = 2*(precision_with*recall_with)/(precision_with+recall_with)

print('\nWITH')
print(f'Precision: {precision_with}')
print(f'Recall: {recall_with}')
print(f'F1: {f1_with}')

print('\nWITHOUT')
print(f'Precision: {precision_without}')
print(f'Recall: {recall_without}')
print(f'F1: {f1_without}')


WITH
Precision: 0.7916666666666666
Recall: 0.57
F1: 0.6627906976744186

WITHOUT
Precision: 0.7878787878787878
Recall: 0.5473684210526316
F1: 0.6459627329192547


In [7]:
u

4.2

In [8]:
#np.mean(time_with)
print(np.mean([float(a) for a in time_with]))
print(np.std([float(a) for a in time_with]))
print(np.mean([float(a) for a in time_without]))
print(np.std([float(a) for a in time_without]))

20.307971428571427
13.996577390792092
15.535921052631583
5.326168606602714


In [9]:
# Opening JSON file
batches =['0','1']
import collections
from collections import defaultdict
divergence, paraphrase, original = [], [],[]
divergence_type = []
time_without, time_with = [],[]
with_, without_, gold_ = defaultdict(list), defaultdict(list),  defaultdict(str)
with_type, without_type = defaultdict(list),  defaultdict(list)
for batch in batches:
    for f_name in glob(f'responses/ced_gpt/gpt_lit_en_pt_ced_{batch}/*.json'):


        f = open(f_name)
        data = json.load(f)
        try:
            if data['language'] == 'Limited':# or data['language'] == 'native':
                continue
        except:
            a=1
        #if data['with_hl'] == 'false':
        #    continue

        #print(data['with_hl'])
        #print(data['totalTimeInMinutes'])
        if data['with_hl'] == 'true':
            time_with.append(data['totalTimeInMinutes'])
        else:
            time_without.append(data['totalTimeInMinutes'])


        #try:
        #    print(data['h28'])
        #except:
        #    continue

        #divergence, paraphrase, original = [], [],[]
        #divergence_type = []
        inds = [a for a in data['sample_indices'].split(',')]
        for id_, ind_ in enumerate(inds):
            ind = str(ind_)
            with open(f'static/examples/for_user_study/gpt_lit_en_pt_ced_{batch}/meta_lbl_{ind}.txt', 'r') as meta:
                meta = meta.readlines()[0]

                try:
                    anno = data[f't{id_+1}']
                except:
                    print('Missing')
                    continue

                #print(f'\nGold: {meta}')
                #print(f'Anno: {anno}')
                #if data['atncheck1'] != data['class2']:
                #    print('Failed first attention check')

                #if data['atncheck2'] != data['t7']:
                #    print('Failed second attention check')

                type_ = data[f'class{id_+1}']
                if 'Major' in type_:# or 'Minor' in type_:     
                    anno_corrected = 'yes'
                else:
                    anno_corrected = 'no'
                #anno_corrected = anno

                if meta == 'ced_gpt':
                    divergence.append(anno_corrected)
                    divergence_type.append(data[f'class{id_+1}'])
                else:
                    #continue
                    original.append(anno_corrected)

                if data['with_hl'] == 'true':
                    with_[f'{ind}_{batch}'].append(anno_corrected)
                    with_type[f'{ind}_{batch}'].append(data[f'class{id_+1}'])
                else:
                    without_[f'{ind}_{batch}'].append(anno_corrected)
                    without_type[f'{ind}_{batch}'].append(data[f'class{id_+1}'])


                #print(f'{ind} {meta}')
                gold_[f'{ind}_{batch}'] = meta

                #if meta == 'chatGPT_paraphrase' and anno=='yes':
                #    with open(f'static/examples/for_user_study/flores_gpt_paraphrase_fr/src_{ind}.txt', 'r') as src:
                #        src = src.readlines()[0]
                #        print(f'\n{src}')

                #    with open(f'static/examples/for_user_study/flores_gpt_paraphrase_fr/tgt_{ind}.txt', 'r') as tgt:
                #        tgt = tgt.readlines()[0]
                #        print(f'\n{tgt}')       

Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing


In [10]:
with_

defaultdict(list,
            {'21_0': ['no',
              'no',
              'no',
              'yes',
              'no',
              'no',
              'no',
              'no',
              'no',
              'no'],
             '11_0': ['no',
              'no',
              'no',
              'no',
              'no',
              'no',
              'no',
              'no',
              'no',
              'no'],
             '2_0': ['yes',
              'no',
              'no',
              'no',
              'no',
              'no',
              'yes',
              'no',
              'yes',
              'no'],
             '13_0': ['no',
              'no',
              'no',
              'no',
              'no',
              'no',
              'no',
              'no',
              'no',
              'no'],
             '1_0': ['yes',
              'no',
              'no',
              'no',
              'no',
              'no',
              '

In [11]:
import random
import numpy as np
from sklearn.metrics import cohen_kappa_score

k = []
for i in range(25):
    labeler1, labeler2 = [],[]

    for annos in with_.values():
        #print(annos)
        [l1, l2] = random.sample(annos, 2)
        labeler1.append(l1)
        labeler2.append(l2)
    k.append(cohen_kappa_score(labeler1, labeler2))
    
print(np.mean(k))
print(np.std(k))

k = []
for i in range(25):
    labeler1, labeler2 = [],[]

    for annos in without_.values():
        [l1, l2] = random.sample(annos, 2)
        labeler1.append(l1)
        labeler2.append(l2)
    k.append(cohen_kappa_score(labeler1, labeler2))
    
print(np.mean(k))
print(np.std(k))

/Users/ebriakou/opt/anaconda3/envs/playground/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


0.5777441262974514
0.11039641965631809
0.44058012410020964
0.12178041525462065


In [13]:
with_flattened = defaultdict(str)
for key, value in with_.items():
    with_flattened[f'{key}_a'] = value[0]
    with_flattened[f'{key}_b'] = value[1]
    with_flattened[f'{key}_c'] = value[2]
    with_flattened[f'{key}_d'] = value[3]
    with_flattened[f'{key}_e'] = value[4]


        
without_flattened = defaultdict(str)
for key, value in without_.items():
    without_flattened[f'{key}_a'] = value[0]
    without_flattened[f'{key}_b'] = value[1]
    without_flattened[f'{key}_c'] = value[2]
    without_flattened[f'{key}_d'] = value[3]
    without_flattened[f'{key}_e'] = value[4]
        
print(len(with_flattened))

250


In [22]:
gold_

defaultdict(str,
            {'21_0': 'original',
             '11_0': 'original',
             '2_0': 'ced',
             '13_0': 'original',
             '1_0': 'ced',
             '22_0': 'original',
             '15_0': 'original',
             '4_0': 'ced',
             '12_0': 'original',
             '19_0': 'original',
             '23_0': 'original',
             '16_0': 'original',
             '8_0': 'ced_gpt',
             '7_0': 'ced_gpt',
             '9_0': 'ced_gpt',
             '5_0': 'ced',
             '18_0': 'original',
             '25_0': 'original',
             '24_0': 'original',
             '6_0': 'ced_gpt',
             '20_0': 'original',
             '3_0': 'ced',
             '17_0': 'original',
             '14_0': 'original',
             '10_0': 'ced_gpt',
             '3_1': 'ced',
             '13_1': 'original',
             '8_1': 'ced_gpt',
             '6_1': 'ced_gpt',
             '25_1': 'original',
             '15_1': 'original',
         

In [34]:
pr, re, f1_ = [], [],[]
samples = 10000
exlcude_easy = True
subsample_size = 250
p_pr, p_re, p_f1 = 0, 0, 0
for i in range(samples):

    subsample = []
    for j in range(subsample_size):
        subsample.append(random.sample(list(with_flattened.keys()), 1)[0])
    
    tp, fn, fp = 0, 0, 0
    for key in subsample:
        sampled = with_flattened[key]
        key_ = '_'.join(key.split('_')[:2])
        if gold_[key_] == 'ced' and exlcude_easy:
            continue
   
        if gold_[key_] == 'ced_gpt' and sampled == 'yes':
            tp +=1
        elif gold_[key_] == 'ced' and sampled == 'yes':
            tp +=1
            
        elif gold_[key_] == 'ced_gpt' and sampled == 'no':
            fn +=1
        elif gold_[key_] == 'ced' and sampled == 'no':
            fn +=1
            
        elif gold_[key_] != 'ced_gpt' and sampled == 'yes':
            fp +=1
        elif gold_[key_] != 'ced' and sampled == 'yes':
            fp +=1

    #print(tp)
    #print(fp)
    #print(fn)
    precision_with = tp/(tp+fp)
    recall_with = tp/(tp+fn)
    f1_with = 2*(precision_with*recall_with)/(precision_with+recall_with)

    #pr.append(precision)
    #re.append(recall)
    #f1_.append(f1)
    
    subsample = []
    for j in range(subsample_size):
        subsample.append(random.sample(list(without_flattened.keys()), 1)[0])
    
    
    tp, fn, fp = 0, 0, 0
    for key in subsample:
        sampled = without_flattened[key]
        key_ = '_'.join(key.split('_')[:2])
        if gold_[key_] == 'ced' and exlcude_easy:
            continue
   
        if gold_[key_] == 'ced_gpt' and sampled == 'yes':
            tp +=1
        elif gold_[key_] == 'ced' and sampled == 'yes':
            tp +=1
            
        elif gold_[key_] == 'ced_gpt' and sampled == 'no':
            fn +=1
        elif gold_[key_] == 'ced' and sampled == 'no':
            fn +=1
            
        elif gold_[key_] != 'ced_gpt' and sampled == 'yes':
            fp +=1
        elif gold_[key_] != 'ced' and sampled == 'yes':
            fp +=1

        #print(gold_[key])
        #print(most_frequent(value))
    precision_without = tp/(tp+fp)
    recall_without = tp/(tp+fn)
    f1_without = 2*(precision_without*recall_without)/(precision_without+recall_without)
    
    if recall_with < recall_without:
        p_re+=1
    #print(f'{precision_with}\t{precision_without}')
    if precision_with < precision_without:
        p_pr+=1
    if f1_with < f1_without:
        p_f1+=1

print(p_pr/samples)
print(p_re/samples)
print(p_f1/samples)

0.4289
0.2788
0.2886


In [25]:
pr, re, f1_ = [], [],[]
samples = 10000
exlcude_easy = False
p_pr, p_re, p_f1 = 0, 0, 0
for i in range(samples):

    tp, fn, fp = 0, 0, 0
    for key, value in with_.items():
        if gold_[key] == 'ced' and exlcude_easy:
            continue
   
        sampled = random.sample(value, 1)[0]
        if gold_[key] == 'ced_gpt' and sampled == 'yes':
            tp +=1
        elif gold_[key] == 'ced' and sampled == 'yes':
            tp +=1
            
        elif gold_[key] == 'ced_gpt' and sampled == 'no':
            fn +=1
        elif gold_[key] == 'ced' and sampled == 'no':
            fn +=1
            
        elif gold_[key] != 'ced_gpt' and sampled == 'yes':
            fp +=1
        elif gold_[key] != 'ced' and sampled == 'yes':
            fp +=1

    #print(tp)
    #print(fp)
    #print(fn)
    precision_with = tp/(tp+fp)
    recall_with = tp/(tp+fn)
    f1_with = 2*(precision_with*recall_with)/(precision_with+recall_with)

    #pr.append(precision)
    #re.append(recall)
    #f1_.append(f1)
    
    tp, fn, fp = 0, 0, 0
    for key, value in without_.items():
        if gold_[key] == 'ced' and exlcude_easy:
            continue

        sampled = random.sample(value, 1)[0]
        if gold_[key] == 'ced_gpt' and sampled == 'yes':
            tp +=1
        elif gold_[key] == 'ced' and sampled == 'yes':
            tp +=1
            
        elif gold_[key] == 'ced_gpt' and sampled == 'no':
            fn +=1
        elif gold_[key] == 'ced' and sampled == 'no':
            fn +=1
            
        elif gold_[key] != 'ced_gpt' and sampled == 'yes':
            fp +=1
        elif gold_[key] != 'ced' and sampled == 'yes':
            fp +=1

        #print(gold_[key])
        #print(most_frequent(value))
    precision_without = tp/(tp+fp)
    recall_without = tp/(tp+fn)
    f1_without = 2*(precision_without*recall_without)/(precision_without+recall_without)
    
    if recall_with < recall_without:
        p_re+=1
    #print(f'{precision_with}\t{precision_without}')
    if precision_with < precision_without:
        p_pr+=1
    if f1_with < f1_without:
        p_f1+=1

print(p_pr/samples)
print(p_re/samples)
print(p_f1/samples)

0.4304
0.2108
0.2693


In [ ]:
def most_frequent(List):
    counter = 0
    num = List[0]
     
    for i in List:
        curr_frequency = List.count(i)
        if(curr_frequency> counter):
            counter = curr_frequency
            num = i
 
    return num

tp, fn, fp = 0, 0, 0
for key, value in without_.items():
    
    if gold_[key] == 'ced_gpt' and most_frequent(value) == 'yes':
        tp +=1
    elif gold_[key] == 'ced' and most_frequent(value) == 'yes':
        tp +=1
    
    elif gold_[key] == 'ced_gpt' and most_frequent(value) == 'no':
        fn +=1
    elif gold_[key] == 'ced' and most_frequent(value) == 'no':
        fn +=1
        
    elif gold_[key] != 'ced_gpt' and most_frequent(value) == 'yes':
        fp +=1
    elif gold_[key] != 'ced' and most_frequent(value) == 'yes':
        fp +=1
        
    #print(gold_[key])
    #print(most_frequent(value))
precision = tp/(tp+fp)
recall = tp/(tp+fn)

print(f'Precision: {precision}')
print(f'Recall: {recall}')
f1 = 2*(precision*recall)/(precision+recall)

print(f'F1: {f1}')

In [ ]:
tp, fn, fp = 0, 0, 0
for key, value in with_.items():
    
    if gold_[key] == 'ced_gpt' and most_frequent(value) == 'yes':
        tp +=1
    elif gold_[key] == 'ced' and most_frequent(value) == 'yes':
        tp +=1
    
    elif gold_[key] == 'ced_gpt' and most_frequent(value) == 'no':
        fn +=1
    elif gold_[key] == 'ced' and most_frequent(value) == 'no':
        fn +=1
        
    elif gold_[key] != 'ced_gpt' and most_frequent(value) == 'yes':
        fp +=1
    elif gold_[key] != 'ced' and most_frequent(value) == 'yes':
        fp +=1
        
    #print(gold_[key])
    #print(most_frequent(value))
precision = tp/(tp+fp)
recall = tp/(tp+fn)

print(f'Precision: {precision}')
print(f'Recall: {recall}')
f1 = 2*(precision*recall)/(precision+recall)

print(f'F1: {f1}')

In [ ]:
k = []
for i in range(100):
    labeler1, labeler2 = [],[]

    for annos in with_type.values():
        #if len(set(annos))==1 and annos[0]=='N/A':
        #    continue
        #annos = [a for a in annos if a!='N/A']
        try:
            [l1, l2] = random.sample(annos, 2)
        except:
            continue
        labeler1.append(l1)
        labeler2.append(l2)
    k.append(cohen_kappa_score(labeler1, labeler2))
    
print(np.mean(k))
print(np.std(k))

k = []
for i in range(100):
    labeler1, labeler2 = [],[]

    for annos in without_type.values():
        #if len(set(annos))==1 and annos[0]=='N/A':
        #    continue
        #annos = [a for a in annos if a!='N/A']
        try:
            [l1, l2] = random.sample(annos, 2)
        except:
            continue
        labeler1.append(l1)
        labeler2.append(l2)
    k.append(cohen_kappa_score(labeler1, labeler2))
    
print(np.mean(k))
print(np.std(k))
print(batch)

In [ ]:
import seaborn as sns
sns.histplot(usefuleness, kde=False)

In [ ]:
sns.histplot(like, kde=False)

In [3]:
usefuleness

[5.0, 4.0, 5.0, 4.0, 4.0, 4.0, 3.0, 5.0, 3.0, 5.0]

In [4]:
like

[4.0, 4.0, 5.0, 4.0, 5.0, 5.0, 3.0, 4.0, 4.0, 5.0]

In [5]:
len(conf)

20